# 67373视频推荐系统

In [1]:
from sklearn.model_selection import train_test_split # pip install scikit-learn
from minisom import MiniSom                          # pip install minisom
import numpy as np                                   # pip install numpy

In [2]:
# 从本地读取数据集
f = open("dataset/dataset.txt", "r")
lines = f.read().splitlines()
dataset = []
for line in lines:
    # arr格式为: [title, cate_name, cate_id, duration, creation_time, video_id]
    arr = line.split(',')
    dataset.append(arr)

## 创建codebook

In [3]:
# 提取数据集每行的前两个元素（即title和cate_name），以字为单位创建codebook
codebook = set()
for data in dataset:
    codebook.update(data[0], data[1])
codebook = list(codebook)
print('codebook长度: {}个字'.format(len(codebook)))
print('codebook内容:\n{}'.format(codebook))

codebook长度: 681个字
codebook内容:
['喜', '雪', '亲', '题', '乎', '牵', 'n', '医', '辑', '探', '画', '所', '价', '荼', '知', '县', '乘', '量', '的', '了', '义', '目', 'f', '带', '情', '娴', '麦', '患', 'M', '毕', '空', '方', '跟', '太', '参', '命', '另', '元', '抖', '学', '痕', '之', '鲸', '剧', '桥', '蛋', '站', '9', '唱', '野', '勒', '匆', 'て', '尘', '阙', '基', '昧', ' ', '币', '底', '主', '么', '硬', '温', '寻', '孤', '通', '诗', 'a', '放', '着', '条', '孩', '，', '英', '利', '車', 'U', '漫', '月', '那', '咪', '盼', '青', '4', '喝', '远', '咖', '亮', 'C', 'い', 'd', '益', '冰', '感', '和', '直', '心', 'T', '想', '工', '至', '公', '其', '怀', '窍', '垣', '遇', '计', '吗', '兰', '局', '戏', '专', '形', '翻', '杂', '爆', '恋', '乐', '万', '彼', '什', '间', '最', '流', '作', '梦', '斗', '灯', '《', 'k', '街', '李', '她', '名', '纯', '夸', '花', '又', '骑', '结', '光', '没', '追', '在', '当', '单', '还', '洋', '路', '现', '进', '贝', '求', '克', '暴', '锦', '梅', 'も', '走', '惑', '忘', '次', 'Y', '8', '都', '城', '业', '场', '鱼', '轮', '长', '隙', '回', '习', '婆', '每', '岁', 'l', '满', '影', '此', '童', '哥', '大', '安', '弦', '劝', '楼', '飞', '藏', '件', '受',

## Bag of Words算法

In [4]:
# 参数inputs可以是字符串也可以是字符串数组
def BoW(inputs, codebook):
    # 初始化返回结果
    # vec是该inputs的one-hot向量表示
    # indices是该inputs在codebook里的索引
    vec = [0] * len(codebook)
    indices = []
    # 处理inputs，如果inputs是数组，则拼接成一个字符串
    # 例如：inputs是["童话镇", "视频-直播回放"]，则拼接成 "童话镇视频-直播回放"
    if (type(inputs) == list):
        inputs = ''.join(inputs)
    # 匹配codebook
    try:
        for char in list(inputs):
            index = codebook.index(char)
            vec[index] = 1
            indices.append(index)
    except:
        print('ValueError: value is not in list')
    return np.array(vec), np.array(indices)

In [5]:
# 测试
# vec, indices = BoW("童话镇", codebook)
# vec, indices = BoW(["童话镇"], codebook)
vec, indices = BoW(["童话镇", "视频-直播回放"], codebook)
print('向量长度为:\n{}'.format(len(vec)))
print('向量表示为:\n{}'.format(vec))
print('索引位置为:\n{}'.format(indices))

向量长度为:
681
向量表示为:
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 

## 创建训练集和测试集

In [6]:
# 首先把数据集中的字符串数据（标题，分类）通过BoW算法用向量表示
x = [] # 用来训练的输入向量
y = [] # 判断训练结果的标签
for data in dataset:
    title = data[0]
    category = data[1]
    vec, indices = BoW([title, category], codebook)
    x.append(vec)
    y.append(title)
x = np.array(x)
y = np.array(y)
print('x的纬度: {}'.format(x.shape))
print('y的纬度: {}'.format(y.shape))

x的纬度: (328, 681)
y的纬度: (328,)


In [7]:
# 划分数据集
x_train, x_test, y_train, y_test = train_test_split(x, y)
print('训练集x的纬度: {}'.format(x_train.shape))
print('训练集y的纬度: {}'.format(y_train.shape))
print('测试集x的纬度: {}'.format(x_test.shape))
print('测试集y的纬度: {}'.format(y_test.shape))

训练集x的纬度: (246, 681)
训练集y的纬度: (246,)
测试集x的纬度: (82, 681)
测试集y的纬度: (82,)


## Self Organizing Map算法

In [8]:
# 初始化一个10x10的SOM
som = MiniSom(10, 10, len(codebook), sigma=1, learning_rate=1)
som.train(x_train, 10000, verbose=True)

 [ 10000 / 10000 ] 100% - 0:00:00 left 
 quantization error: 1.004362150590788


In [9]:
# 查看训练结果
win_map = som.win_map(x_train, return_indices=True)
win_map

defaultdict(list,
            {(7, 6): [0, 9, 49],
             (2, 6): [1, 32],
             (3, 5): [2, 121, 245],
             (4, 8): [3,
              6,
              34,
              41,
              47,
              55,
              60,
              62,
              63,
              65,
              67,
              79,
              92,
              96,
              102,
              103,
              108,
              125,
              126,
              183,
              187,
              195,
              203],
             (7, 7): [4, 39],
             (2, 1): [5, 75],
             (4, 3): [7, 82, 169],
             (0, 7): [8, 155],
             (7, 4): [10, 46],
             (2, 8): [11, 45, 211, 231],
             (9, 8): [12, 105, 136],
             (0, 1): [13, 188, 217],
             (1, 0): [14],
             (0, 6): [15, 135, 219],
             (3, 0): [16],
             (5, 3): [17],
             (6, 8): [18, 100],
             (6, 0): [19],
    

In [10]:
# 测试
test_results = open("test_results/test_results.txt", "w")
for i in range(0, len(x_test)):
    # 从测试集中选取数据
    test_data = x_test[i]
    test_results.write('测试数据: {}\n'.format(y_test[i]))
    # 找出获胜者单元
    winner = som.winner(test_data)
    test_results.write('获胜单元: {}\n'.format(winner))
    # 根据win_map找到该获胜者单元所在的cluster里的其他单元
    cluster = win_map[winner]
    test_results.write('获胜单元所在的cluster: {}\n'.format(cluster))
    # 根据y_train得到这些单元的标签
    recomms = y_train[cluster]
    test_results.write('获胜单元所在的cluster里的标签:\n{}\n\n'.format(recomms))
test_results.close()